# Relatório sobre a competição KDD 2009

## Informações fornecidas pela competição:
Dataset usado: **Small**
- Dataset com 50.000 observações
- 230 atributos(variáveis) sendo: 190 numéricos e 40 categóricos.


## Objetivos a serem desenvolvidos:
 - Fazer a limpeza e transformações necessárias
 - Fazer uma análise e exploração dos dados
 - Testar alguns modelos de algoritmos e verificar qual melhor se adapta ao problema

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

## Carga dos dados

In [31]:
dados = pd.read_table('orange_small_train.data')
churn = pd.read_table('orange_small_train_churn.labels', header=-1)
appetency = pd.read_table('orange_small_train_appetency.labels', header=-1)
upselling = pd.read_table('orange_small_train_upselling.labels', header=-1)

## Verificação da integridade dos dados.

In [32]:
# Informações básicas sobre os dados:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 230 entries, Var1 to Var230
dtypes: float64(191), int64(1), object(38)
memory usage: 87.7+ MB


In [33]:
# Informações básicas sobre o churn:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 1 columns):
0    50000 non-null int64
dtypes: int64(1)
memory usage: 390.7 KB


In [34]:
# Informações básicas sobre o Upselling
upselling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 1 columns):
0    50000 non-null int64
dtypes: int64(1)
memory usage: 390.7 KB


In [35]:
# Informações básicas sobre o appetency
appetency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 1 columns):
0    50000 non-null int64
dtypes: int64(1)
memory usage: 390.7 KB


---
Antes de iniciar as análises podemos verificar que os dados batem com aquilo que foi passado pela competição, não ocorrendo a perda de dados pelo processo de carregamento dos mesmos.  
As variáveis estão nomeadas como **Var1** até **Var230**, o que pode indicar que foi feito dessa forma para preservar a privacidade e segurança dos dados sobre os clientes, mas impossibilita o levantamento de algum tipo de hipótese baseado na análise dos nomes das colunas. 

Obs: As próximas fases de limpeza e transformação dos dados podem ser muito custosas do ponto de vista computacional e por isso os dados de churn, appetency e upselling serão apagados na linha abaixo, e carregados novamente nas fases de criação dos modelos de aprendizagem de máquina.

In [36]:
# liberando memória.
del churn
del upselling
del appetency

## Visualização dos dados

In [37]:
# Visualização primeiros registros
dados.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN


In [38]:
# Visualização dos ultimos registros
dados.tail()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
49995,NaN,NaN,NaN,NaN,NaN,357.0,0.0,NaN,NaN,NaN,...,oslk,EROH7Cg,LM8l689qOp,NaN,NaN,7FJQ,RAYp,F2FyR07IdsN7I,NaN,NaN
49996,NaN,NaN,NaN,NaN,NaN,1078.0,0.0,NaN,NaN,NaN,...,oslk,GfSQowC,LM8l689qOp,NaN,kG3k,FSa2,RAYp,55YFVY9,am7c,NaN
49997,NaN,NaN,NaN,NaN,NaN,2807.0,7.0,NaN,NaN,NaN,...,oslk,dh6qI2t,LM8l689qOp,NaN,ELof,fKCe,RAYp,TCU50_Yjmm6GIBZ0lL_,NaN,NaN
49998,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,oslk,2fF2Oqu,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
49999,NaN,NaN,NaN,NaN,NaN,1694.0,7.0,NaN,NaN,NaN,...,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN


## Informações estatísticas sobre os dados

In [39]:
dados.describe(include='all')

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
count,702.000000,1241.000000,1240.000000,1579.000000,1.487000e+03,44471.000000,44461.000000,0.0,702.000000,1.487000e+03,...,50000,50000,44789,820,23856,50000,50000,50000,21568,0.0
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7,4291,4,1,3,23,7,30,4,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,oslk,catzS2D,LM8l689qOp,4n2X,ELof,FSa2,RAYp,F2FyR07IdsN7I,am7c,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37009,4441,36608,820,11072,8031,35156,32703,11689,NaN
mean,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,6.809496,NaN,48.145299,3.926057e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,40.709951,0.141933,4270.193518,1.275481,6.441259e+05,2685.693668,6.326053,NaN,154.777855,9.280896e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,NaN,0.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,518.000000,0.000000,NaN,4.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,861.000000,7.000000,NaN,20.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,16.000000,0.000000,0.000000,0.000000,1.187425e+05,1428.000000,7.000000,NaN,46.000000,2.628630e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
Com o comando acima podemos visualizar um resumo estatístico dos dados e tirar algumas conclusões:

- Os dados possuem colunas aparentemente com quase sua totalidade de valores faltantes ou inválidos, precisará ser tratado para a utilização de algoritmos de aprendizado de máquina.


In [40]:
# Contagem de quantos valores NaN estão presentes nos dados
dados.isna().sum()

Var1      49298
Var2      48759
Var3      48760
Var4      48421
Var5      48513
Var6       5529
Var7       5539
Var8      50000
Var9      49298
Var10     48513
Var11     48760
Var12     49442
Var13      5539
Var14     48760
Var15     50000
Var16     48513
Var17     48421
Var18     48421
Var19     48421
Var20     50000
Var21      5529
Var22      5009
Var23     48513
Var24      7230
Var25      5009
Var26     48513
Var27     48513
Var28      5011
Var29     49298
Var30     49298
          ...  
Var201    37217
Var202        1
Var203      143
Var204        0
Var205     1934
Var206     5529
Var207        0
Var208      143
Var209    50000
Var210        0
Var211        0
Var212        0
Var213    48871
Var214    25408
Var215    49306
Var216        0
Var217      703
Var218      703
Var219     5211
Var220        0
Var221        0
Var222        0
Var223     5211
Var224    49180
Var225    26144
Var226        0
Var227        0
Var228        0
Var229    28432
Var230    50000
Length: 230, dtype: int6

Como foi confirmado que os dados precisam de tratamento será adotado o seguinte processo:

- Serão eliminadas as colunas que possuirem menos que 60% dos dados válidos
- Os dados das colunas que restarem e forem NaN serão substituidos por 0 (zero)

## Seleção dos dados com 60% dos dados válidos

In [41]:
# Criação de variáveis auxiliares
df_numericos = dados.iloc[:, :190]
df_categoricos = dados.iloc[:, 190:]

In [42]:
# retorna um dataframe com as colunas que possuam ao menos 60% dos dados válidos
df_numericos = df_numericos[[i for i in df_numericos if df_numericos[i].count() / len(df_numericos) >= 0.6]]
df_categoricos = df_categoricos[[i for i in df_categoricos if df_categoricos[i].count() / len(df_categoricos) >= 0.6]]

In [43]:
# quantidade de dados numéricos atuais
df_numericos.shape

(50000, 39)

In [44]:
# quantidade de dados categóricos atuais
df_categoricos.shape

(50000, 28)

In [45]:
# Variaveis restantes
df_numericos.columns

Index(['Var6', 'Var7', 'Var13', 'Var21', 'Var22', 'Var24', 'Var25', 'Var28',
       'Var35', 'Var38', 'Var44', 'Var57', 'Var65', 'Var73', 'Var74', 'Var76',
       'Var78', 'Var81', 'Var83', 'Var85', 'Var109', 'Var112', 'Var113',
       'Var119', 'Var123', 'Var125', 'Var126', 'Var132', 'Var133', 'Var134',
       'Var140', 'Var143', 'Var144', 'Var149', 'Var153', 'Var160', 'Var163',
       'Var173', 'Var181'],
      dtype='object')

In [46]:
# variáveis restantes
df_categoricos.columns

Index(['Var192', 'Var193', 'Var195', 'Var196', 'Var197', 'Var198', 'Var199',
       'Var202', 'Var203', 'Var204', 'Var205', 'Var206', 'Var207', 'Var208',
       'Var210', 'Var211', 'Var212', 'Var216', 'Var217', 'Var218', 'Var219',
       'Var220', 'Var221', 'Var222', 'Var223', 'Var226', 'Var227', 'Var228'],
      dtype='object')

## Tratamento de valores NaN

In [47]:
# substitui os valores NaN por 0
df_numericos.replace(np.nan, 0, inplace=True)
df_categoricos.replace(np.nan, 0, inplace=True)

In [48]:
# Visualização atual dos dados numéricos.
df_numericos.head()

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var134,Var140,Var143,Var144,Var149,Var153,Var160,Var163,Var173,Var181
0,1526.0,7.0,184.0,464.0,580.0,14.0,128.0,166.56,0.0,3570.0,...,69134.0,185.0,0.0,9.0,397579.0,1812252.0,142.0,38418.0,0.0,0.0
1,525.0,0.0,0.0,168.0,210.0,2.0,24.0,353.52,0.0,4764966.0,...,357038.0,0.0,0.0,9.0,278334.0,10439160.0,32.0,238572.0,0.0,0.0
2,5236.0,7.0,904.0,1212.0,1515.0,26.0,816.0,220.08,0.0,5883894.0,...,248932.0,800.0,0.0,36.0,320565.0,9826360.0,206.0,434946.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.08,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1029.0,7.0,3216.0,64.0,80.0,4.0,64.0,200.00,0.0,0.0,...,66046.0,3255.0,0.0,9.0,267162.0,644836.0,2.0,0.0,0.0,0.0


In [49]:
df_numericos.describe(include='all')

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var134,Var140,Var143,Var144,Var149,Var153,Var160,Var163,Var173,Var181
count,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,...,5.000000e+04,50000.000000,50000.000000,50000.000000,5.000000e+04,5.000000e+04,50000.000000,5.000000e+04,50000.000000,50000.000000
mean,1179.759700,6.05514,1111.247920,208.585200,261.168600,3.856080,87.126880,202.007511,0.645000,2.320732e+06,...,3.935276e+05,1228.243700,0.052200,10.430820,2.522753e+05,5.562658e+06,34.915720,4.373827e+05,0.006160,0.550200
std,2566.782817,6.33667,2664.622305,538.420836,673.928257,9.318793,205.367702,115.233463,2.850112,2.958460e+06,...,5.881166e+05,3787.875862,0.610248,11.649139,6.163372e+05,4.523657e+06,95.098069,8.183401e+05,0.125707,2.374479
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-66.880000,0.000000,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
25%,287.000000,0.00000,0.000000,60.000000,75.000000,0.000000,8.000000,166.560000,0.000000,0.000000e+00,...,2.952000e+03,0.000000,0.000000,0.000000,0.000000e+00,4.685830e+05,4.000000,0.000000e+00,0.000000,0.000000
50%,798.000000,7.00000,132.000000,136.000000,170.000000,2.000000,40.000000,200.000000,0.000000,5.807610e+05,...,1.560420e+05,120.000000,0.000000,9.000000,9.898000e+03,5.948800e+06,20.000000,8.107800e+04,0.000000,0.000000
75%,1316.000000,7.00000,1264.000000,212.000000,265.000000,4.000000,104.000000,256.960000,0.000000,4.241100e+06,...,5.491260e+05,1105.000000,0.000000,18.000000,3.067312e+05,1.033293e+07,38.000000,5.254020e+05,0.000000,0.000000
max,131761.000000,140.00000,197872.000000,36272.000000,45340.000000,494.000000,13168.000000,5158.560000,110.000000,1.884690e+07,...,5.735340e+06,520545.000000,18.000000,81.000000,1.693440e+07,1.390780e+07,4862.000000,1.451520e+07,6.000000,49.000000


In [50]:
# visualização dos dados categóricos
df_categoricos.head()

,Var192,Var193,Var195,Var196,Var197,Var198,Var199,Var202,Var203,Var204,...,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var226,Var227,Var228
0,bZkvyxLkBI,RO12,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,dXGu,9_Y1,FbIm,...,sH5Z,cJvF,FzaX,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,xb3V,RAYp,F2FyR07IdsN7I
1,CEat0G8rTN,RO12,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,lg1t,9_Y1,k13i,...,0,0,FzaX,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,fKCe,RAYp,F2FyR07IdsN7I
2,eOQt0GoOh3,AERks4l,taul,1K8T,ffXs,NldASpP,y4g9XoZ,4bTR,9_Y1,MGOA,...,bHR7,UYBR,FzaX,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,Qu4f,02N6s8f,ib5G6X1eUxUn6
3,jg69tYsGvO,RO12,taul,1K8T,ssAy,_ybO0dd,4hMlgkf58mhwh,W8mQ,9_Y1,YULl,...,eKej,UYBR,FzaX,L91KIiz,oslk,CE7uk3u,LM8l689qOp,FSa2,RAYp,F2FyR07IdsN7I
4,IXSgUHShse,RO12,taul,1K8T,uNkU,EKR938I,ThrHXVS,xklU,9_Y1,RVjC,...,H3p7,UYBR,FzaX,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,FSa2,RAYp,F2FyR07IdsN7I


In [51]:
df_categoricos.describe(include='all')

,Var192,Var193,Var195,Var196,Var197,Var198,Var199,Var202,Var203,Var204,...,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var226,Var227,Var228
count,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000,...,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000
unique,362,51,23,4,226,4291,5074,5714,6,100,...,13991,3,23,4291,7,4291,5,23,7,30
top,qFpmfo8zhV,RO12,taul,1K8T,0Xwj,fhk21Ss,r83_sZi,nyZz,9_Y1,RVjC,...,0,cJvF,FzaX,4UxGlow,oslk,catzS2D,LM8l689qOp,FSa2,RAYp,F2FyR07IdsN7I
freq,385,35964,47958,49550,4629,4441,955,198,45233,1819,...,703,25319,40304,4441,37009,4441,36608,8031,35156,32703


In [52]:
# contagen de valores únicos nas colunas dos dados categóricos
df_categoricos.nunique()

Var192      362
Var193       51
Var195       23
Var196        4
Var197      226
Var198     4291
Var199     5074
Var202     5714
Var203        6
Var204      100
Var205        4
Var206       22
Var207       14
Var208        3
Var210        6
Var211        2
Var212       81
Var216     2016
Var217    13991
Var218        3
Var219       23
Var220     4291
Var221        7
Var222     4291
Var223        5
Var226       23
Var227        7
Var228       30
dtype: int64

---
Depois de feita a limpeza dos dados NaN e a exclusão de colunas com poucos dados podemos continuar as análises.


**Sobre os dados numéricos**, podemos notar que algumas colunas possuem variações entre seus valores mais baixos e os mais altos, alguns algoritmos podem fazer predições erradas se os dados não estiverem em escalas de valores semelhantes.Neste caso o ideal é escalonar os dados afim de deixa-los em escalas semelhantes.


**Com relação aos dados categóricos**, algo que pode ser notado é a presença de muitos valores diferentes em algumas colunas, algo que pode vir a ser custoso em termo de processamento e tamanho do dataset, caso seja aplicada técnicas de criação de variáveis dummies(criação de uma nova coluna para um valor categórico, evitando assim, atribuir um valor numérico a ele, o que pode influênciar as previsões de alguns tipos de algoritmos).

In [9]:
teste = pd.get_dummies(df_categoricos)
teste.shape

(50000, 40670)

In [36]:
# mostra a quantidade de valores únicos por coluna
teste.nunique()
# Deleta a variável teste para liberar a memória
#del teste

Var192_0                            2
Var192_0G9vyxdMWg                   2
Var192_0kQTmBU3gb                   2
Var192_0kQqrQsiZt                   2
Var192_0vimfo8zhV                   2
Var192_1GdOj17ejg                   2
Var192_1GdOj1KXzC                   2
Var192_1JGTmBQZiT                   2
Var192_1JGqrQKzJV                   2
Var192_1JGr4RKzJV                   2
Var192_1JGr4RUaLY                   2
Var192_1KSTmBQxul                   2
Var192_1KSqrQK2Mx                   2
Var192_1KSr4RK2Mx                   2
Var192_1PnOj1KVEH                   2
Var192_1PnZptzip9                   2
Var192_1YVTmBftjz                   2
Var192_1YVgUHXZeb                   2
Var192_1YVvyx7IEC                   2
Var192_1fZWq6e2Rv                   2
Var192_1fZWq6wFxs                   2
Var192_1nlTmBtueT                   2
Var192_1qmgUHEuty                   2
Var192_1qmr4RQxul                   2
Var192_25_t0Gdncc                   2
Var192_2AAWq69R_D                   2
Var192_2AArE

No exemplo acima pode ser observado como ficaria o dataset de dados categóricos se for aplicado a criação de variáveis dummies, a quantidade de variáveis aumentaria muito, de 28 para 40.670, o que iria demandar um custo bem mais alto de processamento, além de um tempo muito maior para gerar as predições.

## Criação de variáveis dummies

In [53]:
# Retira as colunas com valores acima de 6000 registros únicos.
df_dummies = df_categoricos[[i for i in df_categoricos if df_categoricos[i].nunique() <= 6000]]

In [54]:
df_dummies = pd.get_dummies(df_dummies)

In [55]:
df_dummies.shape

(50000, 26679)

Foi decidido retirar as colunas com os valores mais extremos, e por isso foi mantido as colunas que possuiam até 6000 registros únicos.  
o dataframe resultante possui um total de 26.679 colunas que representa uma grande economia de processamento em relação ao dataframe sem tratamento.

## Escalonamento dos atributos numéricos

In [56]:
# Criação de um dataframe que receberá todos os valores tratados.
df_final = pd.concat((df_numericos, df_dummies), axis=1)

In [57]:
scaler = StandardScaler()

In [58]:
df_final.iloc[:, :39] = scaler.fit_transform(df_final.iloc[:,:39])

In [59]:
df_final.head()

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var228__URKn_77G3QyQHSVQ2N1RKmtW,Var228__cTCyH95OE93jSkoIBT,Var228_am14IcfM7tWLrUmRT52KtA,Var228_b9qbUNk0dML_Mvi2,Var228_d0LtHjWeaXyArdN4sxU_saXqH,Var228_ib5G6X1eUxUn6,Var228_iyHGyLCEkQ,Var228_n1OBWGkV3fbsHR75taC,Var228_r_7E,Var228_xwM2aC7IdeMC0
0,0.134894,0.149111,-0.347988,0.474382,0.473099,1.088555,0.199026,-0.307618,-0.226309,-0.783240,...,0,0,0,0,0,0,0,0,0,0
1,-0.255092,-0.955581,-0.417042,-0.075379,-0.075927,-0.199178,-0.307388,1.314844,-0.226309,0.826193,...,0,0,0,0,0,0,0,0,0,0
2,1.580298,0.149111,-0.077778,1.863644,1.860501,2.376289,3.549148,0.156835,-0.226309,1.204410,...,0,0,0,0,0,1,0,0,0,0
3,-0.459630,-0.955581,-0.417042,-0.387406,-0.387536,-0.413800,-0.424252,-1.561433,-0.226309,-0.784447,...,0,0,0,0,0,0,0,0,0,0
4,-0.058735,0.149111,0.789895,-0.268538,-0.268827,0.015444,-0.112613,-0.017421,-0.226309,-0.784447,...,0,0,0,0,0,0,0,0,0,0


---

Com o escalonamento dos atributos numéricos, um algoritmo que testa a distância entre seus valores não irá definir um peso maior para um atributo baseado apenas na diferença de valores entre os mesmos.  
Já foi definido um dataframe com os dados numéricos e categóricos já pré-processados.  
Este novo dataframe possui 50.000 observações e 26.718 variáveis para predição.

# Preparação dos modelos

### Preparação dos dados

In [60]:
# apagar os dataframes que não serão usados para liberar a memoria
del dados
del df_categoricos
del df_dummies
del df_numericos


In [61]:
# Separação dos dados de treino e teste
churn = pd.read_table('orange_small_train_churn.labels', header=-1)

x_train, x_test,y_train, y_test = train_test_split(df_final, churn, test_size=0.3, random_state=123456)

auc_churn = []

del churn

## Serão testados os seguintes algoritmos:

- Logistic Regression
- Naive Bayes
- Decision Tree
- Random Forest

# Criação dos Modelos - Target: Churn


---

**EXPLICAÇÃO SOBRE AS MÉTRICAS DE AVALIAÇÃO**

**Classification Report:**  
Mostra um conjunto de métricas, tais como:  
*Precision:* que mostra o número de acertos de cada classe. Como o valor da classe mais assertiva influência o valor da média geral nem sempre esta é uma boa métrica para conjuntos de dados com classes muito desbalanceadas.  
*Recall:* leva em consideração os a quantidade de registros classificados corretamente em ambas as classes.  
*F1 Score:* média entre precisão e recall, ele é melhor utilizado quando o modelo não utiliza probabilidade.

**Confusion Matrix:**  
Com ela é possivel ver a distribuição das classes pelo algoritmo, assim podemos realmente conferir se as previsões não estão tentando seguir algum tipo de tendência.

|Matriz de confusão ||
|-------------------- |--------------- |
|**positivos verdadeiros**|falsos positivos|
|falsos negativos|**negativos verdadeiros**|

**AUC** - Métrica usada pela competição, ela gera uma curva de área formada com as taxas dos valores positivos classificados corretamente e os falsos positivos, ela é usada com conjuntos que estão desbalanceados.\

## Logistic Regression

In [62]:
logistc_model = LogisticRegression()
logistc_model.fit(x_train, y_train)
logistic_predict = logistc_model.predict(x_test)

#Métricas - Relatório de clasificação
print(classification_report(y_test, logistic_predict))

# Confusion matrix
print('\nMATRIZ DE CONFUSÃO\n',confusion_matrix(y_test, logistic_predict))

# AUC
print('\nAUC: ',roc_auc_score(y_test, logistic_predict))


auc_churn.append(['LOGISTIC REGRESSION',roc_auc_score(y_test, logistic_predict)])

             precision    recall  f1-score   support

         -1       0.93      0.99      0.96     13880
          1       0.23      0.03      0.05      1120

avg / total       0.87      0.92      0.89     15000


MATRIZ DE CONFUSÃO
 [[13771   109]
 [ 1087    33]]

AUC:  0.5108056298888431


---

## Naive Bayes


In [63]:
bayes_model = GaussianNB()
bayes_model.fit(x_train, y_train)
bayes_predict = bayes_model.predict(x_test)

# Resumo dos resultados
print(classification_report(y_test, bayes_predict))

# confusion matrix
print('\nMATRIZ DE CONFUSÃO\n', confusion_matrix(y_test, bayes_predict))

print('\nAUC: ', roc_auc_score(y_test, bayes_predict))

auc_churn.append(['NAIVE BAYES', roc_auc_score(y_test, bayes_predict)])

             precision    recall  f1-score   support

         -1       0.93      0.64      0.76     13880
          1       0.08      0.41      0.14      1120

avg / total       0.87      0.62      0.71     15000


MATRIZ DE CONFUSÃO
 [[8904 4976]
 [ 664  456]]

AUC:  0.5243207081103334


---

## Decision Tree

In [64]:
tree_model = DecisionTreeClassifier(criterion='entropy')
tree_model.fit(x_train, y_train)
tree_predict = tree_model.predict(x_test)

print(classification_report(y_test,tree_predict))

# Confusion matrix
print('\nMATRIZ DE CONFUSÃO\n',confusion_matrix(y_test, tree_predict))

print(roc_auc_score(y_test, tree_predict))

auc_churn.append(['DECISION TREE', roc_auc_score(y_test, tree_predict)])

             precision    recall  f1-score   support

         -1       0.93      0.94      0.94     13880
          1       0.16      0.14      0.15      1120

avg / total       0.87      0.88      0.88     15000


MATRIZ DE CONFUSÃO
 [[13027   853]
 [  958   162]]
0.5415937628653766


---

## Random Forest

In [65]:
forest_model = RandomForestClassifier(n_estimators=500)
forest_model.fit(x_train,y_train)
forest_predict = forest_model.predict(x_test)

print(classification_report(y_test, forest_predict))

print('\nMATRIZ DE CONFUSÃO\n', confusion_matrix(y_test, forest_predict))

print('AUC: ',roc_auc_score(y_test, forest_predict))

auc_churn.append(['RANDOM FOREST', roc_auc_score(y_test, forest_predict)])

             precision    recall  f1-score   support

         -1       0.93      1.00      0.96     13880
          1       1.00      0.00      0.00      1120

avg / total       0.93      0.93      0.89     15000


MATRIZ DE CONFUSÃO
 [[13880     0]
 [ 1119     1]]
AUC:  0.5004464285714286


---

# Criação dos Modelos - Target: Appetency


In [69]:
# divisão de variáveis
appetency = pd.read_table('orange_small_train_appetency.labels', header=-1)

x_train, x_test,y_train, y_test = train_test_split(df_final, appetency, test_size=0.3, random_state=123456)

auc_appetency = []

del appetency

## Logistic Regression

In [70]:
logistc_model = LogisticRegression()
logistc_model.fit(x_train, y_train)
logistic_predict = logistc_model.predict(x_test)

#Métricas - Relatório de clasificação
print(classification_report(y_test, logistic_predict))

# Confusion matrix
print('\nMATRIZ DE CONFUSÃO\n',confusion_matrix(y_test, logistic_predict))

# AUC
print('\nAUC: ',roc_auc_score(y_test, logistic_predict))


auc_appetency.append(['LOGISTIC REGRESSION',roc_auc_score(y_test, logistic_predict)])

             precision    recall  f1-score   support

         -1       0.98      1.00      0.99     14735
          1       0.36      0.02      0.03       265

avg / total       0.97      0.98      0.97     15000


MATRIZ DE CONFUSÃO
 [[14728     7]
 [  261     4]]

AUC:  0.5073096401201094


---

## Naive Bayes

In [71]:
bayes_model = GaussianNB()
bayes_model.fit(x_train, y_train)
bayes_predict = bayes_model.predict(x_test)

# Resumo dos resultados
print(classification_report(y_test, bayes_predict))

# confusion matrix
print('\nMATRIZ DE CONFUSÃO\n', confusion_matrix(y_test, bayes_predict))

print('\nAUC: ', roc_auc_score(y_test, bayes_predict))

auc_appetency.append(['NAIVE BAYES', roc_auc_score(y_test, bayes_predict)])

             precision    recall  f1-score   support

         -1       0.98      0.92      0.95     14735
          1       0.03      0.14      0.05       265

avg / total       0.97      0.91      0.94     15000


MATRIZ DE CONFUSÃO
 [[13578  1157]
 [  229    36]]

AUC:  0.5286642636259452


---

## Decision Tree

In [72]:
tree_model = DecisionTreeClassifier(criterion='entropy')
tree_model.fit(x_train, y_train)
tree_predict = tree_model.predict(x_test)

print(classification_report(y_test,tree_predict))

# Confusion matrix
print('\nMATRIZ DE CONFUSÃO\n',confusion_matrix(y_test, tree_predict))

print('\nAUC: ',roc_auc_score(y_test, tree_predict))

auc_appetency.append(['DECISION TREE', roc_auc_score(y_test, tree_predict)])

             precision    recall  f1-score   support

         -1       0.98      0.99      0.98     14735
          1       0.05      0.04      0.05       265

avg / total       0.97      0.97      0.97     15000


MATRIZ DE CONFUSÃO
 [[14538   197]
 [  254    11]]

AUC:  0.5140699528141826


---


## Random Forest

In [73]:
forest_model = RandomForestClassifier(n_estimators=500)
forest_model.fit(x_train,y_train)
forest_predict = forest_model.predict(x_test)

print(classification_report(y_test, forest_predict))

print('\nMATRIZ DE CONFUSÃO\n', confusion_matrix(y_test, forest_predict))

print('\nAUC: ',roc_auc_score(y_test, forest_predict))

auc_appetency.append(['RANDOM FOREST', roc_auc_score(y_test, forest_predict)])

             precision    recall  f1-score   support

         -1       0.98      1.00      0.99     14735
          1       0.00      0.00      0.00       265

avg / total       0.96      0.98      0.97     15000


MATRIZ DE CONFUSÃO
 [[14735     0]
 [  265     0]]

AUC:  0.5


---

# Criação dos Modelos - Target: Upselling

In [75]:
# divisão de variáveis
upselling = pd.read_table('orange_small_train_upselling.labels', header=-1)

x_train, x_test,y_train, y_test = train_test_split(df_final, upselling, test_size=0.3, random_state=123456)

auc_upselling = []

del upselling

## Logistic Regression

In [76]:
logistc_model = LogisticRegression()
logistc_model.fit(x_train, y_train)
logistic_predict = logistc_model.predict(x_test)

#Métricas - Relatório de clasificação
print(classification_report(y_test, logistic_predict))

# Confusion matrix
print('\nMATRIZ DE CONFUSÃO\n',confusion_matrix(y_test, logistic_predict))

# AUC
print('\nAUC: ',roc_auc_score(y_test, logistic_predict))


auc_upselling.append(['LOGISTIC REGRESSION',roc_auc_score(y_test, logistic_predict)])

             precision    recall  f1-score   support

         -1       0.93      0.99      0.96     13865
          1       0.53      0.13      0.20      1135

avg / total       0.90      0.93      0.90     15000


MATRIZ DE CONFUSÃO
 [[13738   127]
 [  993   142]]

AUC:  0.5579751886901859


---

## Naive Bayes

In [77]:
bayes_model = GaussianNB()
bayes_model.fit(x_train, y_train)
bayes_predict = bayes_model.predict(x_test)

# Resumo dos resultados
print(classification_report(y_test, bayes_predict))

# confusion matrix
print('\nMATRIZ DE CONFUSÃO\n', confusion_matrix(y_test, bayes_predict))

print('\nAUC: ', roc_auc_score(y_test, bayes_predict))

auc_upselling.append(['NAIVE BAYES', roc_auc_score(y_test, bayes_predict)])

             precision    recall  f1-score   support

         -1       0.93      0.67      0.78     13865
          1       0.08      0.37      0.14      1135

avg / total       0.86      0.64      0.73     15000


MATRIZ DE CONFUSÃO
 [[9226 4639]
 [ 714  421]]

AUC:  0.5181708132701904


---

## Decision Tree

In [78]:
tree_model = DecisionTreeClassifier(criterion='entropy')
tree_model.fit(x_train, y_train)
tree_predict = tree_model.predict(x_test)

print(classification_report(y_test,tree_predict))

# Confusion matrix
print('\nMATRIZ DE CONFUSÃO\n',confusion_matrix(y_test, tree_predict))

print('\nAUC: ',roc_auc_score(y_test, tree_predict))

auc_upselling.append(['DECISION TREE', roc_auc_score(y_test, tree_predict)])

             precision    recall  f1-score   support

         -1       0.95      0.96      0.95     13865
          1       0.43      0.41      0.42      1135

avg / total       0.91      0.91      0.91     15000


MATRIZ DE CONFUSÃO
 [[13246   619]
 [  665   470]]

AUC:  0.6847260636312078


---

## Random Forest

In [79]:
forest_model = RandomForestClassifier(n_estimators=500)
forest_model.fit(x_train,y_train)
forest_predict = forest_model.predict(x_test)

print(classification_report(y_test, forest_predict))

print('\nMATRIZ DE CONFUSÃO\n', confusion_matrix(y_test, forest_predict))

print('\nAUC: ',roc_auc_score(y_test, forest_predict))

auc_upselling.append(['RANDOM FOREST', roc_auc_score(y_test, forest_predict)])

             precision    recall  f1-score   support

         -1       0.93      1.00      0.96     13865
          1       0.95      0.07      0.12      1135

avg / total       0.93      0.93      0.90     15000


MATRIZ DE CONFUSÃO
 [[13861     4]
 [ 1060    75]]

AUC:  0.532895399470349


---

# Apresentação dos resultados:

Depois de gerar, treinar e testar os 4 modelos escolhidos, o resultado geral das previsões foi de:

In [83]:
print('TARGET - CHURN:\n')
for i in auc_churn:
    print('\t %s \t\t\t %s' % (i[0], i[1]))

print('\nTARGET - APPETENCY:\n')
for i in auc_appetency:
    print('\t %s \t\t\t %s' % (i[0], i[1]))
    
print('\nTARGET - UPSELLING:\n')
for i in auc_upselling:
    print('\t %s \t\t\t %s' % (i[0], i[1]))

TARGET - CHURN:

	 LOGISTIC REGRESSION 			 0.5108056298888431
	 NAIVE BAYES 			 0.5243207081103334
	 DECISION TREE 			 0.5415937628653766
	 RANDOM FOREST 			 0.5004464285714286

TARGET - APPETENCY:

	 LOGISTIC REGRESSION 			 0.5073096401201094
	 NAIVE BAYES 			 0.5286642636259452
	 DECISION TREE 			 0.5140699528141826
	 RANDOM FOREST 			 0.5

TARGET - UPSELLING:

	 LOGISTIC REGRESSION 			 0.5579751886901859
	 NAIVE BAYES 			 0.5181708132701904
	 DECISION TREE 			 0.6847260636312078
	 RANDOM FOREST 			 0.532895399470349
